# dMRI Preprocessing

This note book processes multiband diffusion weighted imaging for tensor-based analyses

In [ ]:
from nipype.pipeline.engine import Workflow, Node, JoinNode, MapNode
from nipype.interfaces.utility import IdentityInterface, Function
from nipype.interfaces.io import SelectFiles, DataSink, FreeSurferSource
from nipype.interfaces.fsl.utils import Reorient2Std
from nipype.interfaces.fsl.epi import ApplyTOPUP, TOPUP
from nipype.interfaces.fsl.preprocess import MCFLIRT, FLIRT
from nipype.interfaces.freesurfer.preprocess import MRIConvert
from nipype.interfaces.freesurfer.model import Binarize
from nipype.interfaces.freesurfer import FSCommand

# MATLAB setup - Specify path to current SPM and the MATLAB's default mode
from nipype.interfaces.matlab import MatlabCommand
MatlabCommand.set_default_paths('~/spm12/toolbox')
MatlabCommand.set_default_matlab_cmd("matlab -nodesktop -nosplash")

# FSL set up- change default file output type
from nipype.interfaces.fsl import FSLCommand
FSLCommand.set_default_output_type('NIFTI_GZ')

# Study-specific variables
project_home = '/moochie/Cat/EmoGrow/dMRI_proc'
output_dir = project_home + '/proc/preprocessing'
workflow_dir = project_home + '/workflows'
raw_dir = '/moochie/R01-EmoGrow/MRI_Data'
phase_encoding_file = project_home + '/misc/acq_params.txt'

#subjects_list = ['1000','1001','1002','1003','1004','1005','1006',
#                 '1007','1008','1009','1011','1012','1015','1016',
#                 '1017','1019','1021','1023','1025','1026', '1027','3000']
subjects_list = open(project_home + '/misc/emogrow_subjects.txt').read().splitlines()

#freesurfer setup
fs_dir = '/moochie/Cat/Aggregate_anats/subjects_dir'
FSCommand.set_default_subjects_dir(fs_dir)

In [ ]:
### Universal nodes

# get subjects list
infosource = Node(IdentityInterface(fields=['subjid']),
                  name='infosource')
infosource.iterables = [('subjid', subjects_list)]

# Sink data of interest
substitutions = [('_subjid_', '')] #output file name substitutions
datasink = Node(DataSink(base_directory = output_dir,
                        container = output_dir,
                        substitutions = substitutions), 
                name='datasink')

## Unwarping workflow

In [ ]:
# Select func dicoms
def cvt_dicom(seqname,studypath,subjid):
    from nipype import logging, config
    config.enable_debug_mode()
    logging.update_logging(config)
    from glob import glob
    from nipype.interfaces.freesurfer import MRIConvert
    from os.path import abspath
    
    subjid = str(subjid)
    dicoms = glob('%s/%s/%s*/MR.*' % (studypath, subjid, seqname))
    print('%s/%s/%s*/MR.*' % (studypath, subjid, seqname))
    dicom_file=dicoms[0]
    
    cvt = MRIConvert()
    cvt.inputs.out_type = 'niigz'
    cvt.inputs.in_file = dicom_file
    cvt.inputs.out_file = seqname[0:13] + '.nii.gz'
    cvt.run()
    
    nii_file = abspath(seqname[0:13] + '.nii.gz')
    return(nii_file)

def sort_pes(pes):
    from nipype import logging, config
    config.enable_debug_mode()
    logging.update_logging(config)
    
    from nipype.interfaces.fsl import Merge, ExtractROI
    from os.path import abspath
    
    eroi = ExtractROI()
    eroi.inputs.t_min = 0
    eroi.inputs.t_size = 1
    
    for i in [0,1]:
        if 'dMRI_dir98_AP' in pes[i]:
            pe0 = pes[i]
            pe_AP = pes[i]
        elif 'dMRI_dir98_PA' in pes[i]:
            pe_PA = pes[i]
    
    eroi.inputs.in_file = pe_AP
    eroi.inputs.roi_file = 'dMRI_AP.nii.gz' 
    eroi.run()
    pe_AP_1 = abspath('dMRI_AP.nii.gz')
    
    eroi.inputs.in_file = pe_PA
    eroi.inputs.roi_file = 'dMRI_PA.nii.gz' 
    eroi.run()
    pe_PA_1 = abspath('dMRI_PA.nii.gz')
        
    
    me = Merge()
    me.inputs.in_files = [pe_AP_1,pe_PA_1]
    me.inputs.dimension='t'
    me.inputs.merged_file = 'merged_pes.nii.gz'
    me.run()
    
    merged_pes = abspath('merged_pes.nii.gz')

    return(merged_pes, pe0)

In [ ]:
# select all dMRI sequences
selectdmri=Node(Function(input_names=['seqname','studypath','subjid'],
                         output_names=['nii_file'], 
                         function=cvt_dicom),
                name='selectdmri')
selectdmri.inputs.studypath = raw_dir
selectdmri.iterables = ('seqname',['dMRI_dir98_AP_14','dMRI_dir98_PA_14'])

# Sort the files for unwarping
sort_pe_list = JoinNode(Function(input_names=['pes'],
                                 output_names=['merged_pes','pe0'],
                                 function=sort_pes), 
                        name='sort_pe_list', joinsource='selectdmri', joinfield='pes')

# actually do the unwarping
topup =Node(TOPUP(encoding_file=phase_encoding_file), name='topup')

apply_topup = Node(ApplyTOPUP(in_index=[2], encoding_file=phase_encoding_file,
                              method='jac', out_corrected='dmri_unwarped.nii.gz'),
                   name='apply_topup')

# reorient unwarped image to MNI space standard
reorientdmri = Node(Reorient2Std(terminal_output='file'),
                    name='reorientdmri')

In [ ]:
prepreprocflow = Workflow(name='prepreprocflow')
prepreprocflow.connect([(infosource,selectdmri, [('subjid','subjid')]),
                        (selectdmri,sort_pe_list, [('nii_file','pes')]),
                        (sort_pe_list,topup, [('merged_pes','in_file')]),
                        (topup, apply_topup, [('out_fieldcoef','in_topup_fieldcoef'), 
                                              ('out_movpar','in_topup_movpar')]),
                        (sort_pe_list, apply_topup, [('pe0','in_files')]),
                        (apply_topup, reorientdmri, [('out_corrected','in_file')]),
                        (reorientdmri, datasink, [('out_file','unwarped_funcs')])
                       ])

prepreprocflow.base_dir = workflow_dir
prepreprocflow.write_graph(graph2use='flat')
prepreprocflow.run('MultiProc', plugin_args={'n_procs': 4, 'memory_gb':10})

## Preprocessing Workflow

This workflow performs the following steps:
* 

In [ ]:
# FreeSurferSource - Data grabber specific for FreeSurfer data
def FSid(subjid):
    from nipype import logging, config
    config.enable_debug_mode()
    logging.update_logging(config)
    
    fs_id = 'E'+str(subjid)
    return(fs_id)

fsid = Node(Function(input_names=['subjid'],
                     output_names=['fs_id'], 
                     function=FSid), 
            name='fsid')

fssource = Node(FreeSurferSource(subjects_dir=fs_dir),
                run_without_submitting=True,
                name='fssource')